In [ ]:
from variant_selection_and_merge_methods import run_experiment

from process_optimizer import __ProcessOptimizerBase

from itertools import product, repeat, chain

import pandas
import numpy
import multiprocessing
import time

In [ ]:
experiment_folder = '../experiment_data'
results_folder = '../experiment_results'

In [ ]:
tree_types = ['loops_only', 'parallel_only', 'xor_choice_only', 'small_combinatory', 'large_combinatory']
resource_types = ['average_resources', 'average_and_time_cost_specialized_resources']
rand_events = ['with_random_events', 'without_random_events']
runs = ['0', '1', '2']
variant_selection = [__ProcessOptimizerBase.ALL_VARIANTS, __ProcessOptimizerBase.REQUIRED_VARIANTS, __ProcessOptimizerBase.MIN_REQUIRED_VARIANTS]
merge_method = [__ProcessOptimizerBase.HIGHEST_COUNT_MERGE, __ProcessOptimizerBase.WEIGHTED_AVERAGE_MERGE, __ProcessOptimizerBase.PARETO_MERGE]
num_of_experiments = 1

In [ ]:
find_missing = pandas.read_excel(io=f'{results_folder}/variant_selection_and_merge_methods_full_run_no_rep_0.xlsx')
find_missing

In [ ]:
missing_combs = list()

for comb in product(find_missing['tree'].unique(), list(find_missing['resource_set'].unique()), list(find_missing['rand_events'].unique()), list(find_missing['version'].unique()), list(find_missing['variant_selection'].unique()), list(find_missing['merging_methods'].unique()), list(find_missing['opt_method'].unique())):
    if len(find_missing.loc[(find_missing['tree'] == comb[0]) & (find_missing['resource_set'] == comb[1]) & (find_missing['rand_events'] == comb[2]) & (find_missing['version'] == comb[3]) & (find_missing['variant_selection'] == comb[4]) & (find_missing['merging_methods'] == comb[5]) & (find_missing['opt_method'] == comb[6])]) < 1:
        variants_select = None
        merge = None

        match comb[4]:
            case 'Min required variants':
                variants_select = __ProcessOptimizerBase.MIN_REQUIRED_VARIANTS
            case 'All variants':
                variants_select = __ProcessOptimizerBase.ALL_VARIANTS
            case 'Required variants':
                variants_select = __ProcessOptimizerBase.REQUIRED_VARIANTS

        match comb[5]:
            case 'Highest count merge':
                merge = __ProcessOptimizerBase.HIGHEST_COUNT_MERGE
            case 'Weighted average merge':
                merge = __ProcessOptimizerBase.WEIGHTED_AVERAGE_MERGE
            case 'Pareto merge':
                merge = __ProcessOptimizerBase.PARETO_MERGE

        missing_combs.append((comb[0], comb[1], comb[2], str(comb[3]), variants_select, merge, comb[6]))

In [ ]:
start_time = time.perf_counter()

pool = multiprocessing.Pool(10)
res_remain = list(chain.from_iterable(pool.starmap(run_experiment, zip(repeat(experiment_folder), repeat(num_of_experiments), missing_combs))))

pool.close()
pool.join()
end_time = time.perf_counter()

In [ ]:
end_time - start_time

In [ ]:
df_remain = pandas.DataFrame(data=res_remain)

In [ ]:
final = find_missing.copy()

for comb in product(find_missing['tree'].unique(), list(find_missing['resource_set'].unique()), list(find_missing['rand_events'].unique()), list(find_missing['version'].unique()), list(find_missing['variant_selection'].unique()), list(find_missing['merging_methods'].unique()), list(find_missing['opt_method'].unique())):
    if len(find_missing.loc[(find_missing['tree'] == comb[0]) & (find_missing['resource_set'] == comb[1]) & (find_missing['rand_events'] == comb[2]) & (find_missing['version'] == comb[3]) & (find_missing['variant_selection'] == comb[4]) & (find_missing['merging_methods'] == comb[5]) & (find_missing['opt_method'] == comb[6])]) < 1:

        if len(target := df_remain.loc[(df_remain['tree'] == comb[0]) & (df_remain['resource_set'] == comb[1]) & (df_remain['rand_events'] == comb[2]) & (df_remain['version'] == str(comb[3])) & (df_remain['variant_selection'] == comb[4]) & (df_remain['merging_methods'] == comb[5]) & (df_remain['opt_method'] == comb[6])]) > 0:

            final = pandas.concat((final, target))
        else:
            final = pandas.concat((final, pandas.DataFrame(data={
                'Unnamed: 0': [None],
                'rep': [0],
                'file': [f'model_{comb[0]}_{comb[1]}_{comb[2]}_{comb[3]}.xlsx'],
                'tree': [comb[0]],
                'resource_set': [comb[1]],
                'rand_events': [comb[2]],
                'version': [comb[3]],
                'variant_selection': [comb[4]],
                'merging_methods': [comb[5]],
                'opt_method': [comb[6]],
                'run_time': [None],
                'time_mean': [None],
                'time_min': [None],
                'time_max': [None],
                'cost_mean': [None],
                'cost_min': [None],
                'cost_max': [None],
            })))

In [ ]:
final.to_excel(f'{results_folder}/variant_selection_and_merge_methods_full_run_no_rep.xlsx', index=False)